In [48]:
import os
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import shutil
import warnings
warnings.filterwarnings('ignore')
import pickle
import glob

In [51]:
list(dataset_dict)

['plots',
 'amazon',
 'dbpedia',
 'agnews',
 'physician_professor',
 'imdb',
 'fakenews',
 'yelp',
 'photographer_journalist',
 'professor_teacher',
 'painter_architect',
 'tweets',
 'spam']

In [52]:
datasets = ['plots',
 'amazon',
 'dbpedia',
 'agnews',
 'physician_professor',
 'imdb',
 'fakenews',
 'yelp',
 'photographer_journalist',
 'professor_teacher',
 'painter_architect',
 'tweets',
 'spam']

In [63]:
cur_fix_results

{'summary': {'before_fix_global_accuracy': 0.5229736706246774,
  'user_input_size': 20,
  'lf_acc_thresh': 0.7,
  'instance_acc_thresh': 0.8,
  'min_non_abstain_thresh': 0.8,
  'dataset_name': 'plots',
  'random_state': 2,
  'confirm_prev_rate': 1.0,
  'complain_fix_rate': 1.0,
  'new_global_accuracy': 0.6809499225606608,
  'global_accuracy_on_valid_data': 0.8427620632279534,
  'new_global_accuracy_on_valid': 0.6809499225606608,
  'valid_global_data_size': 1202,
  'new_valid_global_data_size': 1937,
  'runtimes': {'snorkel_first_run': 0.5167160034179688,
   'snorkel_run_after_fix': 0.8748397827148438,
   'solver_runtime': 0.5098690986633301,
   'repair_time': 2.1462268829345703},
  'optimal_objective_value': 200.0},
 'fix_details': {'original_98': {'rule': rid:98PredicateNode(id=1, pred=RegexPredicate-tag-(\b(?:romantic|date|kiss|wedding|marriage|honeymoon|crush|flirt|passion|intimacy)\b))
       PredicateNode(id=4, pred=keyword_predicate-word-(young), added=True)
           PredicateN

In [60]:
cur_trees

[rid:98PredicateNode(id=1, pred=RegexPredicate-tag-(\b(?:romantic|date|kiss|wedding|marriage|honeymoon|crush|flirt|passion|intimacy)\b))
     PredicateNode(id=4, pred=keyword_predicate-word-(young), added=True)
         PredicateNode(id=10, pred=keyword_predicate-word-(good), added=True)
             PredicateNode(id=16, pred=keyword_predicate-word-(another), added=True)
                 PredicateNode(id=19, pred=keyword_predicate-word-(soldier), added=True)
                     PredicateNode(id=22, pred=keyword_predicate-word-(film), added=True)
                         PredicateNode(id=25, pred=keyword_predicate-word-(dramatization), added=True)
                             LabelNode(id=26, label=0, added=True)
                             LabelNode(id=27, label=-1, added=True)
                         LabelNode(id=24, label=-1, added=True)
                     LabelNode(id=21, label=1, added=True)
                 LabelNode(id=18, label=1, added=True)
             LabelNode(id=12, l

In [61]:
len(cur_trees)

14

In [25]:
with open('raw_data_kdd/', 'rb') as f:
    res = pickle.load(f)

In [26]:
res

{'summary': {'before_fix_global_accuracy': 0.6478,
  'user_input_size': 20,
  'lf_acc_thresh': 0.7,
  'instance_acc_thresh': 0.8,
  'min_non_abstain_thresh': 0.8,
  'dataset_name': 'agnews',
  'random_state': 1,
  'confirm_prev_rate': 1.0,
  'complain_fix_rate': 1.0,
  'new_global_accuracy': 0.6072666666666666,
  'global_accuracy_on_valid_data': 0.7602988928445679,
  'new_global_accuracy_on_valid': 0.6072666666666666,
  'valid_global_data_size': 51122,
  'new_valid_global_data_size': 60000,
  'runtimes': {'snorkel_first_run': 41.36826705932617,
   'snorkel_run_after_fix': 60.17918586730957,
   'solver_runtime': 0.4283130168914795,
   'repair_time': 1.240706205368042},
  'optimal_objective_value': 294.0},
 'fix_details': {'original_0': {'rule': rid:0PredicateNode(id=1, pred=keyword_predicate-word-(profit,stocks,earnings,market,revenue,trade,deal,merger,acquisition,shares))
       PredicateNode(id=4, pred=keyword_predicate-word-(board), added=True)
           PredicateNode(id=10, pred=ke

In [27]:
list(res)

['summary', 'fix_details']

In [28]:
list(res['fix_details'])

['original_0',
 'original_1',
 'original_2',
 'original_3',
 'original_4',
 'original_5',
 'original_6',
 'original_7',
 'original_8',
 'original_9',
 'original_10',
 'original_11',
 'original_12',
 'original_13',
 'original_14',
 'original_15',
 'original_16',
 'original_17',
 'original_18',
 'original_19',
 'original_20']

In [29]:
def max_depth(root):
    if not root:
        return 0
    return 1 + max(max_depth(root.left), max_depth(root.right))

In [30]:
for k,v in res['fix_details'].items():
    print(f"{v['pre_fix_size']}->{v['after_fix_size']}")

3->21
3->17
3->17
3->23
3->15
3->17
3->17
3->15
3->17
3->15
3->15
3->17
3->17
3->19
3->19
3->13
3->11
3->17
3->15
3->15
3->17


In [72]:
def compute_tree_stats(root):
    """
    Given a root node of a binary tree, compute:
    - max depth
    - total number of nodes
    - number of leaf nodes
    - depths of all leaves (for average leaf depth)
    """
    if root is None:
        return {
            "depth": 0,
            "num_nodes": 0,
            "num_leaves": 0,
            "leaf_depths": []
        }

    max_depth = 0
    num_nodes = 0
    num_leaves = 0
    leaf_depths = []

    def dfs(node, depth):
        # print(type(node))
        nonlocal max_depth, num_nodes, num_leaves
        if node is None:
            return
        num_nodes += 1
        max_depth = max(max_depth, depth)
        if node.left is None and node.right is None:
            num_leaves += 1
            leaf_depths.append(depth)
        else:
            dfs(node.left, depth + 1)
            dfs(node.right, depth + 1)

    # print(root)
    dfs(root, 1)
    return {
        "depth": max_depth,
        "num_nodes": num_nodes,
        "num_leaves": num_leaves,
        "leaf_depths": leaf_depths
    }

In [70]:
import numpy as np

def summarize_tree_list(trees):
    """
    Given a list of trees (each with a .root), summarize:
    - number of rules
    - average and max depth
    - average number of nodes
    - average number of leaves
    - average leaf depth
    """
    depths = []
    node_counts = []
    leaf_counts = []
    all_leaf_depths = []

    for tree in trees:
        # print(type(tree.root))
        stats = compute_tree_stats(tree.root)
        depths.append(stats["depth"])
        node_counts.append(stats["num_nodes"])
        leaf_counts.append(stats["num_leaves"])
        all_leaf_depths.extend(stats["leaf_depths"])

    return {
        "num_rules": len(trees),
        "avg_depth": round(np.mean(depths), 2) if depths else 0,
        "max_depth": max(depths) if depths else 0,
        "avg_num_nodes": round(np.mean(node_counts), 2) if node_counts else 0,
        "avg_num_leaves": round(np.mean(leaf_counts), 2) if leaf_counts else 0,
        "avg_leaf_depth": round(np.mean(all_leaf_depths), 2) if all_leaf_depths else 0
    }

In [73]:
summarize_tree_list(cur_trees)

{'num_rules': 14,
 'avg_depth': np.float64(7.5),
 'max_depth': 10,
 'avg_num_nodes': np.float64(17.14),
 'avg_num_leaves': np.float64(9.07),
 'avg_leaf_depth': np.float64(5.21)}

In [74]:
with open('gpt_prefix_tree_size_summaries.pkl', 'rb') as f:
    prefix_tree_sizes = pickle.load(f)

In [75]:
prefix_tree_sizes

{'agnews': {'num_rules': 21,
  'avg_depth': np.float64(2.0),
  'max_depth': 2,
  'avg_num_nodes': np.float64(3.0),
  'avg_num_leaves': np.float64(2.0),
  'avg_leaf_depth': np.float64(2.0)},
 'plots': {'num_rules': 14,
  'avg_depth': np.float64(2.0),
  'max_depth': 2,
  'avg_num_nodes': np.float64(3.0),
  'avg_num_leaves': np.float64(2.0),
  'avg_leaf_depth': np.float64(2.0)},
 'amazon': {'num_rules': 23,
  'avg_depth': np.float64(2.0),
  'max_depth': 2,
  'avg_num_nodes': np.float64(3.0),
  'avg_num_leaves': np.float64(2.0),
  'avg_leaf_depth': np.float64(2.0)},
 'imdb': {'num_rules': 20,
  'avg_depth': np.float64(2.0),
  'max_depth': 2,
  'avg_num_nodes': np.float64(3.0),
  'avg_num_leaves': np.float64(2.0),
  'avg_leaf_depth': np.float64(2.0)},
 'tweets': {'num_rules': 18,
  'avg_depth': np.float64(2.0),
  'max_depth': 2,
  'avg_num_nodes': np.float64(3.0),
  'avg_num_leaves': np.float64(2.0),
  'avg_leaf_depth': np.float64(2.0)},
 'spam': {'num_rules': 16,
  'avg_depth': np.float64(

In [77]:
summaries = []

In [79]:
for s in ['20', '40', '80']:
    for d in datasets:
        for f in glob.glob(f'raw_data_kdd/*{d}_sample_params_{s}-0.7-0.8-0.8-*.pkl'):
            print(f)
            with open(f, 'rb') as cf:
                cur_fix_results = pickle.load(cf)
                cur_trees = [x['rule'] for x in cur_fix_results['fix_details'].values()]
                cur_summary = summarize_tree_list(cur_trees)
                cur_summary['user_input_size'] = s
                cur_summary['dataset'] = d
                summaries.append(cur_summary)

raw_data_kdd/test_run-plots_sample_params_20-0.7-0.8-0.8-2-20250202_113244.pkl
raw_data_kdd/test_run-plots_sample_params_20-0.7-0.8-0.8-6-20250203_222512.pkl
raw_data_kdd/test_run-plots_sample_params_20-0.7-0.8-0.8-4-20250202_113251.pkl
raw_data_kdd/test_run-plots_sample_params_20-0.7-0.8-0.8-7-20250203_222515.pkl
raw_data_kdd/test_run-plots_sample_params_20-0.7-0.8-0.8-1-20250202_113240.pkl
raw_data_kdd/test_run-plots_sample_params_20-0.7-0.8-0.8-3-20250131_112601.pkl
raw_data_kdd/test_run-plots_sample_params_20-0.7-0.8-0.8-4-20250131_112604.pkl
raw_data_kdd/test_run-plots_sample_params_20-0.7-0.8-0.8-12-20250203_222531.pkl
raw_data_kdd/test_run-plots_sample_params_20-0.7-0.8-0.8-10-20250203_222525.pkl
raw_data_kdd/test_run-plots_sample_params_20-0.7-0.8-0.8-2-20250131_023949.pkl
raw_data_kdd/test_run-plots_sample_params_20-0.7-0.8-0.8-2-20250131_112559.pkl
raw_data_kdd/test_run-plots_sample_params_20-0.7-0.8-0.8-3-20250131_023952.pkl
raw_data_kdd/test_run-plots_sample_params_20-0.7-0

In [81]:
df_summary_tree_structures = pd.DataFrame(summaries)

In [82]:
df_summary_tree_structures.to_csv('tree_after_fix_summary.csv', index=False)

In [83]:
summary = df_summary_tree_structures.groupby(['dataset', 'user_input_size']).agg(
    avg_depth_min=('avg_depth', 'min'),
    avg_depth_max=('avg_depth', 'max'),
    avg_depth_std=('avg_depth', 'std'),
    avg_num_nodes_min=('avg_num_nodes', 'min'),
    avg_num_nodes_max=('avg_num_nodes', 'max'),
    avg_num_nodes_std=('avg_num_nodes', 'std')
).reset_index()

In [84]:
summary

,dataset,user_input_size,avg_depth_min,avg_depth_max,avg_depth_std,avg_num_nodes_min,avg_num_nodes_max,avg_num_nodes_std
0,agnews,20,6.76,8.86,0.602577,15.19,18.90,1.023201
1,agnews,40,10.48,12.62,0.692157,26.05,31.48,1.648556
2,agnews,80,13.05,19.14,1.762343,39.48,49.19,3.055186
3,amazon,20,5.57,7.87,0.733912,13.00,16.57,1.297691
4,amazon,40,8.04,11.70,1.023163,21.09,28.65,2.104659
5,amazon,80,10.96,16.61,1.603958,37.09,44.39,2.536194
6,fakenews,20,4.30,5.45,0.322164,8.70,11.30,0.685393
7,fakenews,40,5.50,7.55,0.578972,12.80,17.20,1.317941
8,fakenews,80,6.25,8.65,0.834166,16.50,26.10,2.880446
9,imdb,20,4.65,5.20,0.157553,11.00,12.40,0.464354


In [85]:
# Convert dictionary to DataFrame and rename columns with prefix 'pre_refine_'
pre_refine_df = pd.DataFrame.from_dict(prefix_tree_sizes, orient='index').reset_index()
pre_refine_df.rename(columns={'index': 'dataset'}, inplace=True)
pre_refine_df = pre_refine_df.add_prefix('pre_refine_')
pre_refine_df.rename(columns={'pre_refine_dataset': 'dataset'}, inplace=True)

# Merge with the summary table
merged_summary = pd.merge(summary, pre_refine_df, on='dataset', how='left')

In [86]:
merged_summary

,dataset,user_input_size,avg_depth_min,avg_depth_max,avg_depth_std,avg_num_nodes_min,avg_num_nodes_max,avg_num_nodes_std,pre_refine_num_rules,pre_refine_avg_depth,pre_refine_max_depth,pre_refine_avg_num_nodes,pre_refine_avg_num_leaves,pre_refine_avg_leaf_depth
0,agnews,20,6.76,8.86,0.602577,15.19,18.90,1.023201,21,2.0,2,3.0,2.0,2.0
1,agnews,40,10.48,12.62,0.692157,26.05,31.48,1.648556,21,2.0,2,3.0,2.0,2.0
2,agnews,80,13.05,19.14,1.762343,39.48,49.19,3.055186,21,2.0,2,3.0,2.0,2.0
3,amazon,20,5.57,7.87,0.733912,13.00,16.57,1.297691,23,2.0,2,3.0,2.0,2.0
4,amazon,40,8.04,11.70,1.023163,21.09,28.65,2.104659,23,2.0,2,3.0,2.0,2.0
5,amazon,80,10.96,16.61,1.603958,37.09,44.39,2.536194,23,2.0,2,3.0,2.0,2.0
6,fakenews,20,4.30,5.45,0.322164,8.70,11.30,0.685393,20,2.0,2,3.0,2.0,2.0
7,fakenews,40,5.50,7.55,0.578972,12.80,17.20,1.317941,20,2.0,2,3.0,2.0,2.0
8,fakenews,80,6.25,8.65,0.834166,16.50,26.10,2.880446,20,2.0,2,3.0,2.0,2.0
9,imdb,20,4.65,5.20,0.157553,11.00,12.40,0.464354,20,2.0,2,3.0,2.0,2.0


In [87]:
merged_summary = merged_summary.round(2)

In [89]:
merged_summary.to_csv('tree_depth_summary.csv', index=False)